<a href="https://colab.research.google.com/github/molabokchi/bokchi_open_lab/blob/main/pytorch_hook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. deepdriver experiment 및 run 생성

In [ ]:
pip install deepdriver==0.7.14

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import deepdriver

In [ ]:
deepdriver.setting(http_host="54.180.86.146:9011" ,grpc_host="54.180.86.146:19051")

In [ ]:
deepdriver.login(key="ZmIyNWQxNGJkMzUxYTVjODQ2NjM5NTgzOTM0YTM2OGE2ZmJiY2M2MWMwOWQ0OWFkNjU2YzNkM2UxMjA0YTVkZg==")

True

In [ ]:
def make_exp_name():
  import socket
  from datetime import datetime
  host_name = socket.gethostname()
  date = datetime.today().strftime("%Y%m%d") 

  return "exp" + "_" +"torch_"+host_name +"_"+date

In [ ]:
# experiment init & config hyperparam
deepdriver.init(exp_name= make_exp_name(), 
                config={ 'learning_rate':0.001,'context_size':2 , 'embedding_dim':10})


DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_torch_75e8514b395f_20230201
Run Name=run-2
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_torch_75e8514b395f_20230201/run-2/run/chart


#2.TorchLog class에 멤버함수로 들어갈 내용

In [ ]:
#torch

def add_log_gradients_hook(module, name, prefix,log_freq=0):
  prefix = prefix + name
  
  if not hasattr(module, "_deepdriver_hook_names"):
    module._deepdriver_hook_names = []
      
  for name, parameter in model.named_parameters():
    print(parameter.requires_grad)
    if parameter.requires_grad:
        log_track_grad = log_track_init(log_freq)
        module._deepdriver_hook_names.append("gradients/" + prefix + name)
        _hook_variable_gradient_stats(
            parameter, "gradients/" + prefix + name, log_track_grad
        )

def _hook_variable_gradient_stats( var, name, log_track):
  """Logs a Variable's gradient's distribution statistics next time backward()
  is called on it.
  """

  def _callback(grad, log_track):
      if not log_track_update(log_track):
          return
      log_tensor_stats(grad.data, name)
  
  handle = var.register_hook(lambda grad: _callback(grad, log_track))
  _hook_handles[name] = handle
  return handle

def log_tensor_stats( tensor, name):
 
    """Add distribution statistics on a tensor's elements to the current History entry"""
    # TODO Handle the case of duplicate names.

    if isinstance(tensor, tuple) or isinstance(tensor, list):
        while (isinstance(tensor, tuple) or isinstance(tensor, list)) and (
            isinstance(tensor[0], tuple) or isinstance(tensor[0], list)
        ):
            tensor = [item for sublist in tensor for item in sublist]
        tensor = torch.cat([t.reshape(-1) for t in tensor])

    # checking for inheritance from _TensorBase didn't work for some reason
    if not hasattr(tensor, "shape"):
        cls = type(tensor)
        raise TypeError(f"Expected Tensor, not {cls.__module__}.{cls.__name__}")

    # HalfTensors on cpu do not support view(), upconvert to 32bit
    if isinstance(tensor, torch.HalfTensor):
        tensor = tensor.clone().type(torch.FloatTensor).detach()

    # Sparse tensors have a bunch of implicit zeros. In order to histo them correctly,
    # we have to count them up and add them to the histo ourselves.
    sparse_zeros = None
    if tensor.is_sparse:
        # Have to call this on a sparse tensor before most other ops.
        tensor = tensor.cpu().coalesce().clone().detach()

        backing_values = tensor._values()
        non_zero_values = backing_values.numel()
        all_values = tensor.numel()
        sparse_zeros = all_values - non_zero_values
        tensor = backing_values

    flat = tensor.reshape(-1)

    # For pytorch 0.3 we use unoptimized numpy histograms (detach is new in 0.4)
    if not hasattr(flat, "detach"):
        tensor = flat.cpu().clone().numpy()
        deepdriver.log({name: deepdriver.histogram(seq=tensor)})
        return
    histogram = deepdriver.histogram(seq=tensor)
    deepdriver.visualize(histogram) # 코드 반영시엔 주석 처리할것!
    deepdriver.log({name: histogram})
    

#2. watch.py 에 들어갈 내용

In [ ]:
_global_watch_idx = 0

In [ ]:
def watch(models,idx=None, log_freq=1000):
  
  if not isinstance(models, (tuple, list)):
      models = (models,)
  global _global_watch_idx
  if idx is None:
    idx = _global_watch_idx
  prefix = ""
  for local_idx, model in enumerate(models):
    global_idx = idx + local_idx
    _global_watch_idx += 1
    if global_idx > 0:
      prefix = "graph_%i" % global_idx
    print(model)
    if not isinstance(model, torch.nn.Module):
        raise ValueError(
            "Expected a pytorch model (torch.nn.Module). Received "
            + str(type(model))
        )
    add_log_gradients_hook(model, name="", prefix=prefix, log_freq=log_freq)
      
LOG_TRACK_COUNT, LOG_TRACK_THRESHOLD = range(2)

# 함수

def log_track_init(log_freq):
    """create tracking structure used by log_track_update"""
    l = [0] * 2
    l[LOG_TRACK_THRESHOLD] = log_freq
    return l
def log_track_update(log_track: int) -> bool:
    """count (log_track[0]) up to threshold (log_track[1]), reset count (log_track[0]) and return true when reached"""
    log_track[LOG_TRACK_COUNT] += 1
    if log_track[LOG_TRACK_COUNT] < log_track[LOG_TRACK_THRESHOLD]:
        return False
    log_track[LOG_TRACK_COUNT] = 0
    return True
    
#
   

#3. example

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # noqa: N812
import torch.optim as optim

In [ ]:
#test example
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [
    ([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
    for i in range(len(test_sentence) - 2)
]
_hook_handles ={}



vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

has_cuda = torch.cuda.is_available()

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), deepdriver.config.embedding_dim, deepdriver.config.context_size)
model = model.cuda() if has_cuda else model
optimizer = optim.SGD(model.parameters(), lr=deepdriver.config.learning_rate)
watch(model, log_freq=100)

for i in range(10):
    print("epoch :" + str(i))
    total_loss = 0
    for batch_i, (context, target) in enumerate(trigrams):

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor(
            [word_to_ix[w] for w in context], dtype=torch.long
        )
        context_idxs = context_idxs.cuda() if has_cuda else context_idxs

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        target = torch.tensor([word_to_ix[target]], dtype=torch.long)
        target = target.cuda() if has_cuda else target
        loss = loss_function(log_probs, target)

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
        if batch_i%10 ==0:
          deepdriver.log({"batch_loss": loss.item()})
    losses.append(total_loss)
print(losses)  # The loss decreased ev

NGramLanguageModeler(
  (embeddings): Embedding(97, 10, sparse=True)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)
True
True
True
True
True
epoch :0


Uploading: [./deepdriver/run/5039/chart/gradients/linear2.bias.CHART.json] |██████████████████████████████| [100.0%] [1/1]

Uploading: [./deepdriver/run/5039/chart/gradients/linear2.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1]

Uploading: [./deepdriver/run/5039/chart/gradients/linear1.bias.CHART.json] |██████████████████████████████| [100.0%] [1/1]

Uploading: [./deepdriver/run/5039/chart/gradients/linear1.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1]

...

Uploading: [./deepdriver/run/5039/chart/gradients/linear2.bias.CHART.json] |██████████████████████████████| [100.0%] [1/1]

Uploading: [./deepdriver/run/5039/chart/gradients/linear2.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1]

Uploading: [./deepdriver/run/5039/chart/gradients/linear1.bias.CHART.json] |██████████████████████████████| [100.0%] [1/1]

Uploading: [./deepdriver/run/5039/chart/gradients/linear1.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1]

Uploading: [./deepdriver/run/5039/chart/gradients/embeddings.weight.CHART.json] |██████████████████████████████| [100.0%] [1/1][518.7493965625763, 515.9789156913757, 513.2264277935028, 510.4907069206238, 507.7701859474182, 505.0642397403717, 502.3716070652008, 499.6920738220215, 497.0238583087921, 494.36655497550964]
